# Aplicación de envío rápido de correos. 📧📧📧📨

### Conexión con la base de datos PostGreSQL 🐘

In [27]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import sys, os
import datetime as dt



In [19]:
# Obtener credenciales del entorno
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT') # psycopg2 maneja el string del puerto
DB_DATABASE = os.getenv('DB_DATABASE')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

In [20]:
# --- Función para cargar la consulta SQL desde un archivo ---
def load_sql_query(query_filename, query_dir='sql_queries'):
   
    current_dir = os.getcwd()
    query_path = os.path.join(current_dir, query_dir, query_filename)

    try:
        with open(query_path, 'r', encoding='utf-8') as file:
            sql_query = file.read()
        return sql_query
    except FileNotFoundError:
        print(f"Error: El archivo de consulta '{query_path}' no se encontró.")
        return None
    except Exception as e:
        print(f"Error al leer el archivo de consulta '{query_path}': {e}")
        return None

conn = None
cur = None

try:
    # 1. Establecer la conexión a la base de datos
    print("Intentando conectar a la base de datos...")
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cur = conn.cursor()
    print("¡Conexión a la base de datos establecida correctamente!")

    # 2. Cargar la consulta SQL desde el archivo
    Query = load_sql_query('historialsolicitud.sql')

    if Query:
        print("\n--- Consulta SQL cargada ---")
        print(Query)
        print("---------------------------\n")

        # 3. Ejecutar la consulta
        print("Ejecutando la consulta...")
        cur.execute(Query)
        dfi1 = pd.read_sql_query(Query, conn)
except psycopg2.Error as e:
    print(f"\n¡Error de base de datos! Detalles: {e}")
    print("Asegúrate de que la base de datos esté corriendo y tus credenciales sean correctas.")
except Exception as e:
    print(f"\n¡Ocurrió un error inesperado! Detalles: {e}")
finally:
    # Asegúrate de cerrar el cursor y la conexión
    if cur:
        cur.close()
        print("Cursor cerrado.")
    if conn:
        conn.close()
        print("Conexión a la base de datos cerrada.")

Intentando conectar a la base de datos...
¡Conexión a la base de datos establecida correctamente!

--- Consulta SQL cargada ---
SELECT h.* FROM api.historialsolicitud AS h
---------------------------

Ejecutando la consulta...


C:\Users\Jreyesm\AppData\Local\Temp\ipykernel_22808\2079815226.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfi1 = pd.read_sql_query(Query, conn)


Cursor cerrado.
Conexión a la base de datos cerrada.


### Carga Hojas de Calculo 📊

In [46]:
file_path = r'xlsx_files\Eventos_SIELP.xlsx' 
sheet_name = 'Eventos SIELP' 

df = None 

if os.path.exists(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        print(f"Hoja '{sheet_name}' de '{file_path}' cargada en DataFrame.")
    except Exception as e:
        print(f"Error al cargar Excel: {e}")
else:
    print(f"Error: Archivo no encontrado en '{file_path}'")


Hoja 'Eventos SIELP' de 'xlsx_files\Eventos_SIELP.xlsx' cargada en DataFrame.


### Filtros y estructura del mensaje.

### Configuración con correos electrónicos